In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv('cancer_dataset.csv')

# Features and target
X = df.drop(columns=['cancer'])  # Name of target column
y = df['cancer']  # Name of target column

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
def create_model():
    model = Sequential()
    
    # Input layer
    model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # Hidden layers
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # Output layer with 'linear' activation
    model.add(Dense(1, activation='linear'))
    
    # Compile the model with from_logits=True
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=['accuracy'])
    
    return model

# Create the model
model = create_model()

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_data=(X_test, y_test),
                    verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

# Predict on the test set
y_pred_prob = model.predict(X_test)

# Apply the logistic (sigmoid) function manually
y_pred = tf.nn.sigmoid(y_pred_prob)

# Convert probabilities to class labels
y_pred_class = (y_pred > 0.5).numpy().astype("int32")

# Performance metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Classification Report:\n", classification_report(y_test, y_pred_class))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))
print(f"Accuracy: {accuracy_score(y_test, y_pred_class):.2f}")
